In [1]:

!pip install transformers sentencepiece torch


In [2]:

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

/Users/raluca/myproject/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)

In [4]:

def summarize_romance_text_mbart(text: str, max_tokens: int = 300) -> str:
    tokenizer.src_lang = "ro_RO"   # Romanian input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)

    # Set target language for generation
    generated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"],  # English output
        max_length=max_tokens,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0].strip()

In [6]:

text = """
Al treilea Program | 1968
[…] De curând, am tradus poezie amerindiană (inclusiv silabe “fără sens”, cuvinte distorsionate & muzică) & mi-am explorat propriile surse ancestrale din lumea misticilor, hoţilor & nebunilor evrei.
Cred că în poezie totul este posibil & că încercările noastre anterioare de a o defini, „occidentale”, reprezintă un eşec de percepţie care nu mai trebuie tolerat.
"""

In [7]:

summary = summarize_romance_text_mbart(text)
print("Summary:", summary)

/Users/raluca/myproject/.venv/lib/python3.9/site-packages/transformers/pytorch_utils.py:328: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


Summary: Third Program | 1968 [...] Recently, I have translated Amerindian poetry (including "no-sense" syllables, distorted words & music) & explored my own ancestral sources in the world of mystics, hobbits & Jewish madmen. I believe that in poetry everything is possible & that our earlier attempts to define it as "Western" represent a failure of perception that must no longer be tolerated.
